# Information about Ice raids in February, 2017

By Christian McDonald, with much help from Cody Winchester

Of note: There is problem with converting the .xlsx file as it came from Phil. I had to open it in Excel and `save as` first before csvkit would properly process the contents of the file.

Right now this works with a single csv of the three available.

`ICE - ORR - 4200 released 01Feb2017 and Reinstated with ICE Warrant.xlsx` was opened an a Save as done to rename as `reinstated.xlsx` into data-feb-raw.

This:
- converts the spreadsheet to csv
- parses the csv into to tables
    + First is a list of the people and info about them
    + Second is the list of charges, with booking_id of the person


In [1]:
%%bash
# Convert to csv
in2csv data-feb-raw/reinstated.xlsx > data-feb/reinstated.csv

/Users/christian/anaconda/envs/data/lib/python3.5/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")
/Users/christian/anaconda/envs/data/lib/python3.5/site-packages/openpyxl/workbook/workbook.py:102: UserWarning: Call to deprecated function or class get_active_sheet (Use the .active property).
  def get_active_sheet(self):


In [2]:
%%bash
# this trims crud from the head and tail
count=$(wc -l < data-feb/reinstated.csv | sed 's/ //g'); trim=$(echo $count - 10 | bc); \
tail -n +6 data-feb/reinstated.csv | head -n $trim > data-feb/reinstated-trimmed.csv


In [3]:
%%bash
# report line count, stripping spaces
wc -l < data-feb/reinstated-trimmed.csv | sed 's/ //g'


201


In [4]:
# python imports
import csv
import datetime
import pandas as pd

In [5]:
# helper functions

# This is the name unmangler
def name_unmangler(name_str):
    """Return `LAST,REST` as a tuple"""
    try:
        name_split = name_str.split(',')
        return (name_split[0], name_split[1])
    except:
        return name_str


In [6]:
# scrape spreadsheet data into a python dictionary
with open('data-feb/reinstated-trimmed.csv', 'r') as infile:
    data = csv.reader(infile, delimiter=',')
    data = list(data)
    
    # setting up our people dataframe
    people = []
    
    # setting up our charges dataframe
    charges = []
    
    booking_id_tracker = ''
    
    # set initial defaults    
    new_record = False
    booking_id = None

    # We are working through the data in the csv
    for row in data:
        row = [x.strip() for x in row]

        if new_record:
            # setting vars for each column of the row
            booking_id = row[0]
            booking_id_tracker = booking_id #sets the tracker to this record
            name = row[1]
            name_last = name_unmangler(name)[0]
            name_rest = name_unmangler(name)[1]
            race = row[2]
            sex = row[3]
            jail_custody = row[4]
            age = row[5]
            booking_date = row[6]
            release_date = row[7]
            nativity = row[8]
            
            # adding a row to the people list with vars pulled from current row
            people.append([
                    booking_id,
                    name_last,
                    name_rest,
                    race,
                    sex,
                    jail_custody,
                    age,
                    booking_date,
                    release_date,
                    nativity
                ])
        
            # reset flag to indicate new record
            new_record = False
        
        else:
            # Ensure not blank row, or header or subheader
            if ''.join(row).strip() != '' and row[1].strip() != 'Charge' and row[0] != 'Booking No':
                #setting vars for each column of the row being parsed
                authority = row[0]
                charge_id = row[1]
                charge_description = row[2]
                skip_3 = row[3]
                level = row[4]
                sentence = row[5]
                bond_amount = row[6]
                disp_date = row[7]
                disp_type = row[8]

                charges.append([
                        booking_id_tracker,
                        authority,
                        charge_id,
                        charge_description,
                        level,
                        sentence,
                        bond_amount,
                        disp_date,
                        disp_type
                    ])
        
        # setting the flag at the end of the loop so we know it is
        # a new record.
        # if it's a blank row, the next one is a new record
        if ''.join(row).strip() == '' or row[0] == 'Booking No':
            # set flag
            new_record = True

# at this point I have two dictionaries?
# `people` are all the individual people
# `charges` are all the charges, with first row the booking_id of the person

In [7]:
print(people)

[['1541823', 'RAMIREZ-MARTINEZ', 'ALEJANDRO', 'W', 'M', 'IN CUSTODY', '29', '2015-11-02', '', 'Mexico (Use only when state is unknown)'], ['1601778', 'GONZALEZ', 'WILLIAM', 'W', 'M', 'IN CUSTODY', '25', '2016-01-14', '', 'Distrito Federal, Mexico'], ['1621712', 'MOCTEZUMA-MORENO', 'JOSE R', 'W', 'M', 'RELEASED', '33', '2016-06-12', '2017-02-09', 'Mexico (Use only when state is unknown)'], ['1624441', 'CONTRERAS', 'BETO', 'W', 'M', 'IN CUSTODY', '42', '2016-07-03', '', 'Mexico (Use only when state is unknown)'], ['1625127', 'MORA', 'JOSE EUSTASIO', 'W', 'M', 'IN CUSTODY', '32', '2016-07-09', '', 'Mexico (Use only when state is unknown)'], ['1627668', 'SUSTAITA', 'JUAN MANUEL', 'W', 'M', 'IN CUSTODY', '43', '2016-07-29', '', 'Mexico (Use only when state is unknown)'], ['1628855', 'LOPEZ', 'JULIAN', 'W', 'M', 'IN CUSTODY', '31', '2016-08-07', '', 'Honduras'], ['1628918', 'ESTRADA-FLORES', 'RAMON ALCIDES', 'W', 'M', 'IN CUSTODY', '34', '2016-08-07', '', 'Honduras'], ['1629071', 'NAVA-NAARV

In [8]:
print(charges)

[['1541823', 'NEWART', '36010001', 'INDECENCY W/CHILD SEXUAL CONTACT', 'F2', '', '70000', '', ''], ['1541823', 'ICEW', '4200', 'ICE DETAINER', 'F*', '', '', '', ''], ['1601778', 'NEW', '48010019', 'EVADING ARREST DET W/PREV CONVICTION', 'FS', '', '', '2016-01-15', 'NO CHARGES FILED'], ['1601778', 'NEW', '48010020', 'EVADING ARREST DET W/VEH', 'F3', '7 years', '', '', ''], ['1601778', 'NEW', '54040011', 'DRIVING WHILE INTOXICATED 3RD OR MORE', 'F3', '7 years', '', '', ''], ['1601778', 'NEW', '29990042', 'CRIMINAL MISCHIEF >=$100<$750', 'MB', '', '', '2016-01-14', 'NO CHARGES FILED'], ['1601778', 'NEW', '35990014', 'POSS CS PG1<1G (FS)', 'FS', '', '', '2016-01-14', 'NO CHARGES FILED'], ['1601778', 'NEW', '29990027', 'CRIM MISCHIEF IMPAIR/INTERRUPT PUB SERVICE', 'FS', '', '', '', ''], ['1601778', 'ICEW', '4200', 'ICE DETAINER', 'F*', '', '', '', ''], ['1601778', 'REIN', '54040011', 'DRIVING WHILE INTOXICATED 3RD OR MORE', 'F3', '', '', '2017-01-18', 'DISMISSAL'], ['1621712', 'NEW', '54040